# Linear agent, 4 moving obstacles, Time-Varying constraints, radar

In [3]:
clear
yalmip clear
close all
warning off

## The agent

First we load a demo agent:

In [4]:
N = 30;     % prediction horizon
Ts = 0.25;  % sampling time
agent = optiplan.LinearAgent.demo2D('PredictionHorizon', N, 'SamplingTime', Ts)

agent = 

  LinearAgent with properties:

                 A: [4x4x30 optiplan.AgentSignal]
                 B: [4x2x30 optiplan.AgentSignal]
                 f: [4x1x30 optiplan.AgentSignal]
                 C: [2x4x30 optiplan.AgentSignal]
                 D: [2x2x30 optiplan.AgentSignal]
                 g: [2x1x30 optiplan.AgentSignal]
                nx: 4
                nu: 2
                ny: 2
                 N: 30
                 X: [4x1x31 optiplan.AgentSignal]
                 U: [2x1x30 optiplan.AgentSignal]
                 Y: [2x1x30 optiplan.AgentSignal]
              Size: [2x1x30 optiplan.AgentSignal]
    ConstraintsFun: []
      ObjectiveFun: []


Tell the agent that the position reference should be treated as a time-varying parameter:

In [5]:
agent.Y.Reference = 'parameter';

Since we will be plotting the agent as a planar object, we set its width and height to one:

In [6]:
agent.Size.Value = [1; 1];

To choose between MIQP or Time-Varying Constraints approach, we need to set variable `MixedInteger` to true/false:

In [7]:
MixedInteger = false;

## The Obstacle

Then we create 4 static obstacles, set their size as constant, visibility and position as parameters:

In [8]:
obstacles = optiplan.Obstacle(agent, 4);
for i = 1:length(obstacles)
    obstacles(i).Size.Value = [3; 2];          % all have fixed size
    obstacles(i).Visible.Value = 'parameter';  % in case of radar, all have time-varying visibility
    obstacles(i).Position.Value = 'parameter'; % all have floating position
end

and minimal separation gab between the agent and the obstacle:

In [9]:
minsep = agent.Size.Value;

## The Planner

Next, we create a planner which will be responsible for optimizing the control inputs for the agent:

In [10]:
planner = optiplan.Planner(agent, obstacles, 'MinSeparation', minsep,...
    'solver', 'gurobi', 'MixedInteger', MixedInteger)

planner = 

  Planner with properties:

      Parameters: [1x1 struct]
    MixedInteger: 0


## The Simulator

To make closed-loop simulation, we create Simulator:

In [11]:
psim = optiplan.Simulator(planner)

psim = 

  Simulator with properties:

       Results: [1x1 struct]
    Parameters: [1x1 struct]


Before starting the simulation, we need to define number of simulation steps, intial point and values for time-varying parameters:

In [12]:
Nsim = 350; % number of simulation steps
x0 = [0; 0; 0; 0]; % initial point
psim.Parameters.Agent.Y.Reference = psim.circularTrajectory(Nsim, 'Radius', 10, 'Loops', 2);

For each obstacle, we set its trajectory:

In [13]:
% obstacles follow a circular trajectory
psim.Parameters.Obstacles(1).Position.Value = psim.circularTrajectory(Nsim, 'Radius', 10, 'Loops', 3,...
    'Center', [0;0], 'InitPoint', pi);
psim.Parameters.Obstacles(2).Position.Value = psim.circularTrajectory(Nsim, 'Radius', 10, 'Loops', 1,...
    'Center', [0;0], 'InitPoint', pi/2);
psim.Parameters.Obstacles(3).Position.Value = psim.circularTrajectory(Nsim, 'Radius', 2, 'Loops', 5,...
    'Center', [0;10]);
psim.Parameters.Obstacles(4).Position.Value = psim.circularTrajectory(Nsim, 'Radius', 3, 'Loops', 5,...
    'Center', [0;-10], 'InitPoint', pi/2);

Then we must define radar properties:

In [14]:
% radar detector: returns true if the obstacle is in the radar's range
RadarRadius = 5;
radar_detector = @(apos, opos, osize) psim.circularRadar(RadarRadius, apos, opos, osize);

where `apos` is agent position, `opos` is obstacle position and `osize` is obstacle size.

When done, we can run the simulation:

In [15]:
psim.run(x0, Nsim, 'RadarDetector', radar_detector);

Before plotting the results, we define function which will plot the radar range around the agent:

In [16]:
% radar plotter plots the radar range w.r.t. current position of the agent
radar_plotter = @(apos) viscircles(apos', RadarRadius);

To plot the results, launch:

In [17]:
psim.plot('RadarDetector', radar_detector, 'RadarPlotter', radar_plotter,...
    'Axis', [-15 15 -15 15], 'Reference', true, 'Trail', true,...
    'Predictions', true, 'PredSteps', 10, 'Constraints', true);

where '`Axis`' sets dimension of both axis, '`Reference`' enable/disable showing the reference, '`Predictions`' enable/disable showing predicted positions of the agent, '`PredSteps`' defines number of displayed predictions, '`Trail`' enable/disable showing realized trajectory of agent and 'Constraints' enable/disable showing the constraints.

## Notes & hints

**Hint**: use the `Simulator/circularTrajectory` helper to automatically create a trajectory over `Nsim` steps which follows specified circular trajectory:

In [18]:
Nsim = 20;
r = optiplan.Simulator.circularTrajectory(Nsim, 'Radius', 5, 'Loops', 1)

r =

  Columns 1 through 7

   -5.0000   -4.7553   -4.0451   -2.9389   -1.5451    0.0000    1.5451
   -0.0000   -1.5451   -2.9389   -4.0451   -4.7553   -5.0000   -4.7553

  Columns 8 through 14

    2.9389    4.0451    4.7553    5.0000    4.7553    4.0451    2.9389
   -4.0451   -2.9389   -1.5451         0    1.5451    2.9389    4.0451

  Columns 15 through 21

    1.5451    0.0000   -1.5451   -2.9389   -4.0451   -4.7553   -5.0000
    4.7553    5.0000    4.7553    4.0451    2.9389    1.5451    0.0000
